# Modèle N-grammes (unigrammes, bigrammes)

Modèle de langue (de base) basé sur l'estimé du maximum de vraisemblance (MLE)
MLE + OOV (les mots de fréquence 1  dans l'entrainement sont considérés comme inconnus)

In [1]:
import nltk

In [2]:
from nltk.corpus import brown
brown_words = brown.words()
fdist = nltk.FreqDist(w.lower() for w in brown_words)
print(brown_words)
sentences = brown.sents()
print('nb sentences', len(sentences))
print('vocabulary', len(set(brown_words)))
total = 0
for word in fdist:
    total += fdist[word]
print('nb of tokens', total)
print(len(brown_words))
print('fact ', fdist['fact'])
print('unicorn', fdist['unicorn'])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]
nb sentences 57340
vocabulary 56057
nb of tokens 1161192
1161192
fact  447
unicorn 0


In [3]:
# on considère les mots de fréquence 1 comme ne faisant pas partie du vocabulaire et donc on augmente le nombre d'OOV
OOV = 0
for w in fdist:
    if fdist[w]==1:    
        OOV+=1
oovp = OOV/total        
print('OOV count:',OOV,'- OOV prob:',oovp)

OOV count: 22010 OOV prob: 0.018954660383468023


In [4]:
test = ['a','fact','about','the','unicorn','is','that','it','does','not','exist','.']

print('Unigrams for example sentence')
for word in test:
    if word in fdist:
        print(word,'- unigram count:',fdist[word],'- unigram prob:', fdist[word]/total)
    else:
        print(word,'- unigram count:',0,'- unigram prob:',0)

Unigrams for example sentence
a - unigram count: 23195 - unigram prob: 0.019975163452727887
fact - unigram count: 447 - unigram prob: 0.00038494925903726515
about - unigram count: 1815 - unigram prob: 0.001563049004815741
the - unigram count: 69971 - unigram prob: 0.06025790739171472
unicorn - unigram count: 0 - unigram prob: 0
is - unigram count: 10109 - unigram prob: 0.008705709305610097
that - unigram count: 10594 - unigram prob: 0.009123383557585654
it - unigram count: 8760 - unigram prob: 0.0075439720563007665
does - unigram count: 485 - unigram prob: 0.00041767425197555613
not - unigram count: 4610 - unigram prob: 0.003970058353829513
exist - unigram count: 59 - unigram prob: 5.0809857456820236e-05
. - unigram count: 49346 - unigram prob: 0.042495986882444936


In [5]:
probabilities = []
index = 0
print(test)
for word in test:
    if (word in fdist) and (fdist[word]>1):
        probabilities.append(fdist[word]/total) # unigram
    else: # on remplace les mots inconnus en entrainement par la probabilite d'un OOV
        probabilities.append(oovp) ## OOV/total
    
print('Unigram prob:',probabilities)
final_probability = probabilities[0]
for prob in probabilities[1:]:
    final_probability = final_probability * prob
print('Sentence Prob:',final_probability)

['a', 'fact', 'about', 'the', 'unicorn', 'is', 'that', 'it', 'does', 'not', 'exist', '.']
Unigram prob: [0.019975163452727887, 0.00038494925903726515, 0.001563049004815741, 0.06025790739171472, 0.018954660383468023, 0.008705709305610097, 0.009123383557585654, 0.0075439720563007665, 0.00041767425197555613, 0.003970058353829513, 5.0809857456820236e-05, 0.042495986882444936]
Sentence Prob: 2.945012433555463e-29


In [6]:
bigrams = nltk.bigrams(brown_words)
cfd = nltk.ConditionalFreqDist(bigrams)
l =list(nltk.bigrams(test))
print(l)
print(cfd)
for (word1,word2) in l:
    if word2 in cfd[word1]:
        bigram_count = cfd[word1][word2]
        print(word1, word2, bigram_count)
    else: print(word1, word2, 0)

[('a', 'fact'), ('fact', 'about'), ('about', 'the'), ('the', 'unicorn'), ('unicorn', 'is'), ('is', 'that'), ('that', 'it'), ('it', 'does'), ('does', 'not'), ('not', 'exist'), ('exist', '.')]
<ConditionalFreqDist with 56057 conditions>
a fact 9
fact about 2
about the 325
the unicorn 0
unicorn is 0
is that 227
that it 329
it does 62
does not 226
not exist 12
exist . 11


In [7]:
probabilities = []
index = 0
previous_word = ''
# Notez qu'on aurait dû utiliser des <S> et </S> normalement pour avoir une probabilité complète et correcte 
# de la phrase
for word in test:
    if (previous_word in fdist) and (word in cfd[previous_word]):
        print(previous_word, word)
        bigram_count = cfd[previous_word][word]
        previous_count = fdist[previous_word]
        probabilities.append(bigram_count/previous_count)
        print(previous_word, word, bigram_count/previous_count)
    elif (word in fdist) and (fdist[word]>1):
       probabilities.append(fdist[word]/total) # unigrammes - une sorte de backoff "maison"
    else:
        probabilities.append(OOV/total) # prise en compte des OOV (UNK)
    index += 1
    previous_word = word
    
print('Bigram prob:',probabilities)
final_probability = probabilities[0]
for prob in probabilities[1:]:
    final_probability = final_probability * prob
print('Sentence Prob:',final_probability)

a fact
a fact 0.00038801465833153696
fact about
fact about 0.0044742729306487695
about the
about the 0.1790633608815427
is that
is that 0.02245523790681571
that it
that it 0.031055314328865394
it does
it does 0.007077625570776256
does not
does not 0.465979381443299
not exist
not exist 0.0026030368763557484
exist .
exist . 0.1864406779661017
Bigram prob: [0.019975163452727887, 0.00038801465833153696, 0.0044742729306487695, 0.1790633608815427, 0.018954660383468023, 0.008705709305610097, 0.02245523790681571, 0.031055314328865394, 0.007077625570776256, 0.465979381443299, 0.0026030368763557484, 0.1864406779661017]
Sentence Prob: 1.1437120989610328e-21
